In [1]:
from iragca.matplotlib import Styles
import matplotlib.pyplot as plt
from PIL import Image
import seaborn as sns
import torch
from torchvision import transforms

from lib.architectures import SimpleCNN, DoubleModelApproach
from lib.config import Directories
from lib.data import (
    PlantDocDiseaseDetection, 
    PlantVillageDiseaseDetection,
    PlantDocSymptomIdentification, 
    PlantVillageSymptomIdentification,
    CombinedPlantVillageDataset,
    CombinedPlantDocDataset,
    DiamosDiseaseDetection,
    DiamosSymptomIdentification,
    CombinedDiamosDataset,
)
from torch.utils.data import DataLoader
from sklearn.metrics import f1_score, accuracy_score, classification_report
from tqdm import tqdm


plt.style.use(Styles.ML.value)

model_name = "disease_detection_model"


IMAGE_SIZE = (32, 32)
transform_pipeline = transforms.Compose(
    [
        transforms.Resize(IMAGE_SIZE),
        transforms.ToTensor(),
    ]
)
KAGGLE_DATA_PATH = Directories.EXTERNAL_DATA_DIR.value / "kagglehub"

plantvillage = PlantVillageDiseaseDetection(data_path=KAGGLE_DATA_PATH / "plantvillage", transforms=transform_pipeline)
plantdoc = PlantDocDiseaseDetection(data_path=KAGGLE_DATA_PATH / "plantdoc", transforms=transform_pipeline)
plantdoc_si = PlantDocSymptomIdentification(data_path=KAGGLE_DATA_PATH / "plantdoc", transforms=transform_pipeline)
plantvillage_si = PlantVillageSymptomIdentification(data_path=KAGGLE_DATA_PATH / "plantvillage", transforms=transform_pipeline)
combined_plantvillage = CombinedPlantVillageDataset(data_path=KAGGLE_DATA_PATH / "plantvillage", transforms=transform_pipeline)
combined_plantdoc = CombinedPlantDocDataset(data_path=KAGGLE_DATA_PATH / "plantdoc", transforms=transform_pipeline)
diamos = DiamosDiseaseDetection(data_path=KAGGLE_DATA_PATH / "diamos", transforms=transform_pipeline)
diamos_si = DiamosSymptomIdentification(data_path=KAGGLE_DATA_PATH / "diamos", transforms=transform_pipeline)
combined_diamos = CombinedDiamosDataset(data_path=KAGGLE_DATA_PATH / "diamos", transforms=transform_pipeline)

In [3]:
plantdoc_loader = DataLoader(plantdoc, batch_size=32, shuffle=True)
plantvillage_loader = DataLoader(plantvillage, batch_size=32, shuffle=True)
plantdoc_si_loader = DataLoader(plantdoc_si, batch_size=32, shuffle=True)
plantvillage_si_loader = DataLoader(plantvillage_si, batch_size=32, shuffle=True)
combined_plantvillage_loader = DataLoader(combined_plantvillage, batch_size=32, shuffle=True)
diamos_loader = DataLoader(diamos, batch_size=32, shuffle=True)
combined_plantdoc_loader = DataLoader(combined_plantdoc, batch_size=32, shuffle=True)
diamos_si_loader = DataLoader(diamos_si, batch_size=32, shuffle=True)
combined_diamos_loader = DataLoader(combined_diamos, batch_size=32, shuffle=True)

In [3]:
model = SimpleCNN(channels=3, output_dim=1)
model.load_state_dict(torch.load(Directories.MODELS_DIR.value / f"{model_name}.pth"))


def test(model, data_loader):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()
    all_preds = []
    all_labels = []
    all_outputs = []
    THRESHOLD = 0.5
    with torch.no_grad():
        for images, labels in tqdm(data_loader, desc=f"Evaluating Dataset {data_loader.dataset.__class__.__name__}"):
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            preds = (outputs >= THRESHOLD).long()

            all_outputs.extend(outputs.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    f1 = f1_score(all_labels, all_preds)
    accuracy = accuracy_score(all_labels, all_preds)

    return f1, accuracy


In [ ]:

f1, accuracy = test(model, plantdoc_loader)
print(f"F1 Score: {f1:.4f}")
print(f"Accuracy: {accuracy:.4f}")

Evaluating Dataset PlantDocDiseaseDetection: 100%|██████████| 92/92 [00:27<00:00,  3.29it/s]

F1 Score: 0.8954
Accuracy: 0.8392


In [5]:
f1, accuracy = test(model, plantvillage_loader)
print(f"F1 Score: {f1:.4f}")
print(f"Accuracy: {accuracy:.4f}")

Evaluating Dataset PlantVillageDiseaseDetection: 100%|██████████| 1698/1698 [00:32<00:00, 52.47it/s]


F1 Score: 0.9901
Accuracy: 0.9856


In [4]:
f1, accuracy = test(model, diamos_loader)
print(f"F1 Score: {f1:.4f}")
print(f"Accuracy: {accuracy:.4f}")

Evaluating Dataset DiamosDiseaseDetection: 100%|██████████| 94/94 [03:48<00:00,  2.43s/it]

F1 Score: 0.9918
Accuracy: 0.9837


## Symptom Identification

In [2]:
symptom_identifier = SimpleCNN(channels=3, output_dim=12)
symptom_identifier.load_state_dict(torch.load(Directories.MODELS_DIR.value / f"symptom_identification_model.pth"))

<All keys matched successfully>

In [3]:
def test_si(model, data_loader):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()
    all_preds = []
    all_labels = []
    all_outputs = []
    with torch.no_grad():
        for images, labels in tqdm(data_loader, desc=f"Evaluating Dataset {data_loader.dataset.__class__.__name__}"):
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            preds = outputs.argmax(dim=1)

            all_outputs.extend(outputs.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    return all_labels, all_preds

In [ ]:
all_labels, all_preds = test_si(symptom_identifier, plantdoc_si_loader)

Evaluating Dataset PlantDocSymptomIdentification: 100%|██████████| 66/66 [00:16<00:00,  4.01it/s]


In [10]:
f1 = f1_score(all_labels, all_preds, average='weighted')
accuracy = accuracy_score(all_labels, all_preds)

print(f"F1 Score: {f1:.4f}")
print(f"Accuracy: {accuracy:.4f}")

F1 Score: 0.7692
Accuracy: 0.7569


In [11]:
print(classification_report(all_labels, all_preds))

              precision    recall  f1-score   support

           0       0.83      0.83      0.83       769
           1       0.84      0.80      0.82       238
           2       0.00      0.00      0.00         0
           3       0.68      0.68      0.68       130
           4       0.03      1.00      0.06         2
           5       0.74      0.67      0.71        91
           6       0.80      0.52      0.63        54
           7       0.75      0.56      0.64        79
           8       0.77      0.77      0.77       223
           9       0.83      0.61      0.70        93
          10       0.72      0.74      0.73       415
          11       0.00      0.00      0.00         0

    accuracy                           0.76      2094
   macro avg       0.58      0.60      0.55      2094
weighted avg       0.79      0.76      0.77      2094



/home/iragca/Documents/github/DS413-final-project/.venv/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/iragca/Documents/github/DS413-final-project/.venv/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/iragca/Documents/github/DS413-final-project/.venv/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _

In [5]:
all_labels, all_preds = test_si(symptom_identifier, plantvillage_si_loader)

Evaluating Dataset PlantVillageSymptomIdentification: 100%|██████████| 1226/1226 [00:22<00:00, 54.17it/s]


In [6]:
f1 = f1_score(all_labels, all_preds, average='weighted')
accuracy = accuracy_score(all_labels, all_preds)

print(f"F1 Score: {f1:.4f}")
print(f"Accuracy: {accuracy:.4f}")

F1 Score: 0.9302
Accuracy: 0.9305


In [7]:
print(classification_report(all_labels, all_preds))

              precision    recall  f1-score   support

           0       0.96      0.78      0.86      6970
           1       0.99      0.99      0.99      5507
           2       0.99      0.98      0.98      5357
           3       0.99      0.96      0.97      2887
           4       0.97      0.85      0.91      1676
           5       0.97      0.87      0.92       952
           6       0.98      0.88      0.93       373
           7       0.96      0.95      0.96      1801
           8       0.97      0.97      0.97      1467
           9       0.92      0.92      0.92       630
          10       0.83      0.98      0.90     10492
          11       0.98      0.96      0.97      1109

    accuracy                           0.93     39221
   macro avg       0.96      0.92      0.94     39221
weighted avg       0.94      0.93      0.93     39221



In [8]:
total_params = sum(p.numel() for p in symptom_identifier.parameters())
print(f"Total parameters: {total_params}")

Total parameters: 268908


In [5]:
all_labels, all_preds = test_si(symptom_identifier, diamos_si_loader)

Evaluating Dataset DiamosSymptomIdentification: 100%|██████████| 93/93 [03:34<00:00,  2.31s/it]


In [6]:
f1 = f1_score(all_labels, all_preds, average='weighted')
accuracy = accuracy_score(all_labels, all_preds)
print(f"F1 Score: {f1:.4f}")
print(f"Accuracy: {accuracy:.4f}")

F1 Score: 0.8919
Accuracy: 0.8853


In [7]:
print(classification_report(all_labels, all_preds))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.00      0.00      0.00         0
           2       1.00      0.69      0.81        54
           3       0.00      0.00      0.00         0
           4       0.90      0.95      0.93      2025
           6       0.00      0.00      0.00         0
           7       0.00      0.00      0.00         0
           8       0.00      0.00      0.00         0
           9       0.00      0.00      0.00         0
          10       0.90      0.75      0.82       884

    accuracy                           0.89      2963
   macro avg       0.28      0.24      0.26      2963
weighted avg       0.90      0.89      0.89      2963



/home/iragca/Documents/github/DS413-final-project/.venv/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/iragca/Documents/github/DS413-final-project/.venv/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/iragca/Documents/github/DS413-final-project/.venv/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _

## Combined Identification and Detection

In [4]:
test_comb = test_si

combined_classifier = SimpleCNN(channels=3, output_dim=13)
combined_classifier.load_state_dict(torch.load(Directories.MODELS_DIR.value / f"combined_identification_model.pth"))

<All keys matched successfully>

In [ ]:
all_labels, all_preds = test_comb(combined_classifier, combined_plantdoc_loader)

In [7]:
f1 = f1_score(all_labels, all_preds, average='weighted')
accuracy = accuracy_score(all_labels, all_preds)

print(f"F1 Score: {f1:.4f}")
print(f"Accuracy: {accuracy:.4f}")

F1 Score: 0.7897
Accuracy: 0.7754


In [8]:
print(classification_report(all_labels, all_preds))

              precision    recall  f1-score   support

           0       0.82      0.84      0.83       769
           1       0.80      0.74      0.77       238
           2       0.00      0.00      0.00         0
           3       0.87      0.75      0.81       130
           4       0.02      1.00      0.04         2
           5       0.76      0.56      0.65        91
           6       0.80      0.44      0.57        54
           7       0.66      0.72      0.69        79
           8       0.79      0.75      0.77       223
           9       0.71      0.61      0.66        93
          10       0.69      0.70      0.69       415
          11       0.00      0.00      0.00         0
          12       0.88      0.84      0.86       822

    accuracy                           0.78      2916
   macro avg       0.60      0.61      0.56      2916
weighted avg       0.81      0.78      0.79      2916



/home/iragca/Documents/github/DS413-final-project/.venv/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/iragca/Documents/github/DS413-final-project/.venv/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/iragca/Documents/github/DS413-final-project/.venv/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _

In [18]:
labels_for_combined = []
preds_for_combined = []

for label, pred in zip(all_labels, all_preds):
    if label == combined_plantdoc.CLASS_MAP['healthy']:
        labels_for_combined.append(0)
        if pred != combined_plantdoc.CLASS_MAP['healthy']:
            preds_for_combined.append(1)
        else:
            preds_for_combined.append(0)

    else:
        labels_for_combined.append(1)
        if pred != combined_plantdoc.CLASS_MAP['healthy']:
            preds_for_combined.append(1)
        else:
            preds_for_combined.append(0)

accuracy_score(labels_for_combined, preds_for_combined)

0.9242112482853223

In [19]:
f1_score(labels_for_combined, preds_for_combined)

0.947717057014431

In [20]:
total_params = sum(p.numel() for p in combined_classifier.parameters())
print(f"Total parameters: {total_params}")

Total parameters: 269037


## PlantVillage

In [5]:
all_labels, all_preds = test_comb(combined_classifier, combined_plantvillage_loader)

Evaluating Dataset CombinedPlantVillageDataset: 100%|██████████| 1698/1698 [00:40<00:00, 42.18it/s]


In [6]:
f1 = f1_score(all_labels, all_preds, average='weighted')
accuracy = accuracy_score(all_labels, all_preds)

print(f"F1 Score: {f1:.4f}")
print(f"Accuracy: {accuracy:.4f}")

F1 Score: 0.9475
Accuracy: 0.9478


In [7]:
print(classification_report(all_labels, all_preds))

              precision    recall  f1-score   support

           0       0.94      0.80      0.87      6970
           1       0.99      0.99      0.99      5507
           2       0.99      0.99      0.99      5357
           3       0.99      0.97      0.98      2887
           4       0.97      0.89      0.93      1676
           5       0.96      0.93      0.94       952
           6       0.88      0.95      0.91       373
           7       0.94      0.94      0.94      1801
           8       0.98      0.97      0.97      1467
           9       0.88      0.91      0.89       630
          10       0.86      0.95      0.90     10492
          11       0.97      0.98      0.98      1109
          12       0.98      0.98      0.98     15084

    accuracy                           0.95     54305
   macro avg       0.95      0.94      0.94     54305
weighted avg       0.95      0.95      0.95     54305



In [8]:
labels_for_combined = []
preds_for_combined = []

for label, pred in zip(all_labels, all_preds):
    if label == combined_plantdoc.CLASS_MAP['healthy']:
        labels_for_combined.append(0)
        if pred != combined_plantdoc.CLASS_MAP['healthy']:
            preds_for_combined.append(1)
        else:
            preds_for_combined.append(0)

    else:
        labels_for_combined.append(1)
        if pred != combined_plantdoc.CLASS_MAP['healthy']:
            preds_for_combined.append(1)
        else:
            preds_for_combined.append(0)

accuracy_score(labels_for_combined, preds_for_combined)

0.9905349415339287

In [9]:
f1_score(labels_for_combined, preds_for_combined)

0.9934437102987321

## DiaMOS

In [10]:
all_labels, all_preds = test_comb(combined_classifier, combined_diamos_loader)

Evaluating Dataset CombinedDiamosDataset: 100%|██████████| 94/94 [03:32<00:00,  2.26s/it]


In [11]:
f1 = f1_score(all_labels, all_preds, average='weighted')
accuracy = accuracy_score(all_labels, all_preds)

print(f"F1 Score: {f1:.4f}")
print(f"Accuracy: {accuracy:.4f}")

F1 Score: 0.8705
Accuracy: 0.8666


In [12]:
print(classification_report(all_labels, all_preds))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.00      0.00      0.00         0
           2       1.00      0.72      0.84        54
           3       0.00      0.00      0.00         0
           4       0.89      0.94      0.91      2025
           5       0.00      0.00      0.00         0
           6       0.00      0.00      0.00         0
           7       0.00      0.00      0.00         0
           8       0.00      0.00      0.00         0
           9       0.00      0.00      0.00         0
          10       0.88      0.71      0.79       884
          12       0.59      0.67      0.63        43

    accuracy                           0.87      3006
   macro avg       0.28      0.25      0.26      3006
weighted avg       0.88      0.87      0.87      3006



/home/iragca/Documents/github/DS413-final-project/.venv/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/iragca/Documents/github/DS413-final-project/.venv/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/iragca/Documents/github/DS413-final-project/.venv/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _

In [13]:
labels_for_combined = []
preds_for_combined = []

for label, pred in zip(all_labels, all_preds):
    if label == combined_plantdoc.CLASS_MAP['healthy']:
        labels_for_combined.append(0)
        if pred != combined_plantdoc.CLASS_MAP['healthy']:
            preds_for_combined.append(1)
        else:
            preds_for_combined.append(0)

    else:
        labels_for_combined.append(1)
        if pred != combined_plantdoc.CLASS_MAP['healthy']:
            preds_for_combined.append(1)
        else:
            preds_for_combined.append(0)

accuracy_score(labels_for_combined, preds_for_combined)

0.9886892880904857

In [14]:
f1_score(labels_for_combined, preds_for_combined)

0.9942567567567567

# Double Model

In [6]:
db_model = DoubleModelApproach(
    disease_detection_state=Directories.MODELS_DIR.value / "disease_detection_model.pth",
    symptom_identifier_state=Directories.MODELS_DIR.value / "symptom_identification_model.pth",
)

all_preds = []
all_labels = []

for images, labels in tqdm(combined_plantvillage_loader, desc="Evaluating Double Model on Combined PlantVillage Dataset"):
    preds = db_model(images)
    all_preds.extend(preds.numpy())
    all_labels.extend(labels.numpy())

print(classification_report(all_labels, all_preds))

Evaluating Double Model on Combined PlantVillage Dataset: 100%|██████████| 1698/1698 [00:57<00:00, 29.62it/s]

              precision    recall  f1-score   support

           0       0.95      0.77      0.85      6970
           1       0.99      0.99      0.99      5507
           2       0.99      0.98      0.98      5357
           3       0.99      0.95      0.97      2887
           4       0.97      0.85      0.90      1676
           5       0.97      0.86      0.91       952
           6       0.98      0.88      0.93       373
           7       0.94      0.92      0.93      1801
           8       0.97      0.97      0.97      1467
           9       0.87      0.84      0.85       630
          10       0.81      0.96      0.88     10492
          11       0.97      0.95      0.96      1109
          12       0.98      0.97      0.97     15084

    accuracy                           0.94     54305
   macro avg       0.95      0.92      0.93     54305
weighted avg       0.94      0.94      0.94     54305



In [7]:
f1 = f1_score(all_labels, all_preds, average='weighted')
accuracy = accuracy_score(all_labels, all_preds)

print(f"F1 Score: {f1:.4f}")
print(f"Accuracy: {accuracy:.4f}")

F1 Score: 0.9360
Accuracy: 0.9360


In [8]:
labels_for_combined = []
preds_for_combined = []

for label, pred in zip(all_labels, all_preds):
    if label == combined_plantdoc.CLASS_MAP['healthy']:
        labels_for_combined.append(0)
        if pred != combined_plantdoc.CLASS_MAP['healthy']:
            preds_for_combined.append(1)
        else:
            preds_for_combined.append(0)

    else:
        labels_for_combined.append(1)
        if pred != combined_plantdoc.CLASS_MAP['healthy']:
            preds_for_combined.append(1)
        else:
            preds_for_combined.append(0)

accuracy_score(labels_for_combined, preds_for_combined)

0.9856182671945493

In [9]:
f1_score(labels_for_combined, preds_for_combined)

0.9900526027536841

## PlantDoc

In [10]:
all_labels = []
all_preds = []

for images, labels in tqdm(combined_diamos_loader, desc="Evaluating Double Model on Combined Diamos Dataset"):
    preds = db_model(images)
    all_preds.extend(preds.numpy())
    all_labels.extend(labels.numpy())

print(classification_report(all_labels, all_preds))

Evaluating Double Model on Combined Diamos Dataset: 100%|██████████| 94/94 [03:38<00:00,  2.32s/it]

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.00      0.00      0.00         0
           2       1.00      0.69      0.81        54
           3       0.00      0.00      0.00         0
           4       0.90      0.95      0.92      2025
           6       0.00      0.00      0.00         0
           7       0.00      0.00      0.00         0
           8       0.00      0.00      0.00         0
           9       0.00      0.00      0.00         0
          10       0.87      0.75      0.81       884
          12       0.12      0.02      0.04        43

    accuracy                           0.87      3006
   macro avg       0.26      0.22      0.23      3006
weighted avg       0.88      0.87      0.87      3006




/home/iragca/Documents/github/DS413-final-project/.venv/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/iragca/Documents/github/DS413-final-project/.venv/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/iragca/Documents/github/DS413-final-project/.venv/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  

In [11]:
accuracy = accuracy_score(all_labels, all_preds)
f1 = f1_score(all_labels, all_preds, average='weighted')

print(f"F1 Score: {f1:.4f}")
print(f"Accuracy: {accuracy:.4f}")

F1 Score: 0.8726
Accuracy: 0.8713


In [12]:
labels_for_combined = []
preds_for_combined = []

for label, pred in zip(all_labels, all_preds):
    if label == combined_plantdoc.CLASS_MAP['healthy']:
        labels_for_combined.append(0)
        if pred != combined_plantdoc.CLASS_MAP['healthy']:
            preds_for_combined.append(1)
        else:
            preds_for_combined.append(0)

    else:
        labels_for_combined.append(1)
        if pred != combined_plantdoc.CLASS_MAP['healthy']:
            preds_for_combined.append(1)
        else:
            preds_for_combined.append(0)

accuracy_score(labels_for_combined, preds_for_combined)

0.9836992681304059

In [13]:
f1_score(labels_for_combined, preds_for_combined)

0.9917799027008891

## DiaMOS

In [14]:
all_labels = []
all_preds = []

for images, labels in tqdm(combined_diamos_loader, desc="Evaluating Double Model on Combined Diamos Dataset"):
    preds = db_model(images)
    all_preds.extend(preds.numpy())
    all_labels.extend(labels.numpy())

print(classification_report(all_labels, all_preds))

Evaluating Double Model on Combined Diamos Dataset: 100%|██████████| 94/94 [03:46<00:00,  2.41s/it]

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.00      0.00      0.00         0
           2       1.00      0.69      0.81        54
           3       0.00      0.00      0.00         0
           4       0.90      0.95      0.92      2025
           6       0.00      0.00      0.00         0
           7       0.00      0.00      0.00         0
           8       0.00      0.00      0.00         0
           9       0.00      0.00      0.00         0
          10       0.87      0.75      0.81       884
          12       0.12      0.02      0.04        43

    accuracy                           0.87      3006
   macro avg       0.26      0.22      0.23      3006
weighted avg       0.88      0.87      0.87      3006




/home/iragca/Documents/github/DS413-final-project/.venv/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/iragca/Documents/github/DS413-final-project/.venv/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/iragca/Documents/github/DS413-final-project/.venv/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  

In [15]:
accuracy = accuracy_score(all_labels, all_preds)
f1 = f1_score(all_labels, all_preds, average='weighted')

print(f"F1 Score: {f1:.4f}")
print(f"Accuracy: {accuracy:.4f}")

F1 Score: 0.8726
Accuracy: 0.8713


In [16]:
labels_for_combined = []
preds_for_combined = []

for label, pred in zip(all_labels, all_preds):
    if label == combined_plantdoc.CLASS_MAP['healthy']:
        labels_for_combined.append(0)
        if pred != combined_plantdoc.CLASS_MAP['healthy']:
            preds_for_combined.append(1)
        else:
            preds_for_combined.append(0)

    else:
        labels_for_combined.append(1)
        if pred != combined_plantdoc.CLASS_MAP['healthy']:
            preds_for_combined.append(1)
        else:
            preds_for_combined.append(0)

accuracy_score(labels_for_combined, preds_for_combined)

0.9836992681304059

In [17]:
f1_score(labels_for_combined, preds_for_combined)

0.9917799027008891